In [ ]:
# !pip install pandas
# !pip install openai
# !pip install python-dotenv
import pandas as pd
import openai
from dotenv import load_dotenv
import os
from tqdm import tqdm
import multiprocessing
import shutil

In [ ]:
load_dotenv()

In [ ]:
openai.api_key = "Your_OpenAI_Key"

# Copying the data to the ../data/generated folder using python
shutil.copy('source_path', 'target_path')


# Creating a global lock for file writing
lock = multiprocessing.Lock()




def get_response(system_prompt, user_prompt):
    """Get the response from the OpenAI API."""
    response = openai.ChatCompletion.create(
              model="gpt-3.5-turbo-16k",
            # model="gpt-4",
              messages=[{"role": "system", "content": system_prompt},
                        {"role": "user", "content": user_prompt},
                        ])

    message = response["choices"][0]["message"]["content"]
    return message


def generate(judgement_text):
    """Generate the response from the OpenAI API."""

    # Give general instructions in the system prompt
    system_prompt = """Your task is to generate nuanced, perspective-based summaries for given legal judgment texts from Indian Supreme Court cases. A 'perspective-based summary' aims to provide a multi-faceted understanding of the judgment. It should include a critical analysis of legal reasoning, implications for future cases, the impact on the parties involved, and broader emotional, social, or political implications. This goes beyond simply summarizing the judgment's facts and findings."""

    # Give more specific instructions in the user prompt and also provide the judgement text
    user_prompt = """Consider the following Indian Supreme Court case judgement text:
    ```
    {}
    ```
    Generate a perspective-based summary for the above judgment text in the perspective of:

    1. The Prosecution Attorney: Your summary should cover these areas:
        - Legal Strengths: Identify legal precedents, statutes, or rules that the judgment confirms or establishes in favor of the prosecution.
        - Evidentiary Gains: Discuss how the judgment impacts the admissibility or weight of key evidence presented by the prosecution.
        - Judicial Reasoning: Comment on the judge's reasoning that aligns with or strengthens the prosecution's arguments.
        Additionally, feel free to consider other significant factors not listed here that impact the prosecution's case.

    2. The Defense Attorney: Your summary should cover these areas:
        - Legal Strengths: Identify legal precedents, statutes, or rules that the judgment confirms or establishes in favor of the defense.
        - Evidentiary Gains: Discuss elements in the judgment that question the admissibility or reliability of the prosecution's evidence.
        - Judicial Reasoning: Highlight portions of the judge's reasoning that are favorable to the defense or offer grounds for an appeal.
        Also, you should incorporate any other crucial aspects not explicitly mentioned that could affect the defense's perspective.
    Note:
    - The summaries should offer nuanced insights specific to each role and not merely serve as mirror opposites.
    - Keep the tone academic and the perspective in the third person.
    - Format your response as a single JSON object with two properties: 'prosecution' and 'defense.' Each should encapsulate the relevant perspectives in a detailed, coherent manner and should be at least 150 words long each.
    Example Response: (This is just an example and not to be used as a template or copied)
    {{
      "prosecution": "From the viewpoint of the prosecution attorney, this judgment consolidates the legal statute that interest on accrual basis, though not realised, is to be included in the net wealth of the assessee, even if the accounts are maintained on a cash basis. This adherence to the provisions of the Wealth Tax Act and the strict interpretation of 'net wealth' strengthens the prosecution's reliance on statutory law. Evidently, the Court has given weightage to the accrued interest on the assessee's money lending investments as important evidence. This could potentially influence similar future cases where assets are not confined to cash. The judge's reasoning aligns with and bolsters the prosecution's arguments, as it endorses consistent statutory interpretation and doesn't allow the choice of accounting system to circumvent the provisions of the Wealth Tax Act. The judgment confirms the importance of valuing all economic activities and liabilities on the valuation date, irrespective of whether they have been realized in cash or not, offering potential advantages in exposing hidden or unrealised assets.",
      "defense": "From the defense attorney's point of view, this judgment raises crucial legal questions about the applicability of the Wealth Tax Act when the assessee maintains accounts on a cash basis. While no direct legal precedent seems to be in favor of the defendant, the contradictions between the rulings of different High Courts indicate room for contesting the legal interpretation. This judgment does not significantly question the admissibility of the prosecution's evidence but underlines the different High Court stances which might be used to challenge the interpretation of the key evidence - the accrued interest. The judge's reasoning acknowledges the apparent contradictions between high court rulings and in doing so, recognizes the need for a uniform interpretational approach. Thus, the defense could argue for an appeal based on the divergent opinions of different High Courts and the need for a more nuanced interpretation of the statute considering the variability and specifics of accounting systems."
    }}
    """.format(judgement_text)

    return get_response(system_prompt, user_prompt)


def generate_and_save(args):
    index, judgement, filepath = args
    try:
        new_text = generate(judgement)
        # Save the result immediately
        with lock:
            df = pd.read_csv(filepath)
            df.at[index, 'GeneratedText'] = new_text
            df.to_csv(filepath, index=False)
        return (index, new_text)
    except Exception as e:
        return (index, None)


def generate_and_save_parallelly(filepath):

    try:
        # Loading the CSV into a DataFrame
        df = pd.read_csv(filepath)

        if 'Target_Column' not in df.columns:
            df['Target_Column'] = None

        to_generate = [(index, row['Text'], filepath) for index, row in df.iterrows() if pd.isnull(row['Target_Column']) or row['Target_Column'] == ""]

        with multiprocessing.Pool(processes=multiprocessing.cpu_count()) as pool:
            list(tqdm(pool.imap(generate_and_save, to_generate), total=len(to_generate), desc="Generating text"))

        # Checking if all rows have been generated
        df = pd.read_csv(filepath)
        if df['Target_Column'].isnull().sum() == 0:
            print("All rows have been generated.")

    except KeyboardInterrupt:
        print("Process interrupted by user. Exiting...")


if __name__ == "__main__":
    generate_and_save_parallelly("target_path")